# INSTALAR REPORTLAB

In [1]:
%pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install untangle

Note: you may need to restart the kernel to use updated packages.


# IMPORTAR LIBRERIAS

In [43]:
import os
import xml.etree.ElementTree as ET
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import utils
import untangle


import pandas as pd

## Lectura de XML

In [46]:
def get_info_from_xml(xml_path):
    doc = untangle.parse(xml_path)
    
    clashtest = doc.exchange.batchtest.clashtests.clashtest
    nombre_archivo = clashtest["name"]
    tolerance = clashtest["tolerance"]
    
    summary = clashtest.summary
    total = summary["total"]
    activas = summary["active"]
    resueltas = summary["resolved"]

    clash_results = clashtest.clashresults
    
    df = pd.DataFrame(columns=['Nombre', 'Imagen', 'Distancia', 'Estado', 'Grid', 'Puntos', 'Fecha', 'Objetos'])
    
    conflictos = []
    i = 0
    for clashresult in clash_results.clashresult:
        # Datos del campo "clashresult"
        nombre_conflicto = clashresult['name']
        imagen_filename = clashresult['href']
        distancia = clashresult['distance']
        
        result_status = clashresult.resultstatus.cdata
        gridlocation = clashresult.gridlocation.cdata
        
        # Datos del campo "clashpoint"
        clashpoint = clashresult.clashpoint.pos3f
        x, y, z = clashpoint['x'], clashpoint['y'], clashpoint['z']

        puntos = [x, y, z]
        
        # Fecha
        anno, mes, dia = clashresult.createddate.date['year'], clashresult.createddate.date['month'], clashresult.createddate.date['day']
        fecha = f'{dia}/{mes}/{anno}'
        
        # Objetos en conflicto
        clashobjects = clashresult.clashobjects
        objetos = []
        
        for clashobject in clashobjects.clashobject:
            value = clashobject.objectattribute.value.cdata
            layer = clashobject.layer.cdata
            
            smarttags = clashobject.smarttags
            tag_nombre = smarttags.smarttag[0].value.cdata
            tag_tipo = smarttags.smarttag[1].value.cdata
            
            objetos.append([value, layer, tag_nombre, tag_tipo])
        #print (i, nombre_conflicto, imagen_filename, distancia, result_status, gridlocation, puntos, fecha, objetos)
        
        
        conflictos.append([nombre_conflicto, imagen_filename, distancia, result_status, gridlocation, puntos, fecha, objetos])
        i += 1
        
        df.loc[i] = [nombre_conflicto, imagen_filename, distancia, result_status, gridlocation, puntos, fecha, objetos]
    
    return nombre_archivo, tolerance, total, activas, resueltas, df
        
        
xml_path = './content/01/2.- ARQ_PUE VS EST.xml'
pdf_output = '/pdf_1.pdf'
output_folder = '/output'

nombre_archivo, tolerance, total, activas, resueltas, archivo_df = get_info_from_xml(xml_path)
print("Nombre: ", nombre_archivo, "\t Tolerancia: ", tolerance, "\t Total: ", total, "\t Activas: ", activas, "\t Resueltas: ", resueltas)
display(archivo_df)

Nombre:  2.- ARQ_PUE VS EST 	 Tolerancia:  0.001 	 Total:  22 	 Activas:  21 	 Resueltas:  1


,Nombre,Imagen,Distancia,Estado,Grid,Puntos,Fecha,Objetos
1,Conflicto1,2.- ARQ_PUE VS EST_files\cd010001.jpg,-0.130,Activo,3'-C : Nivel 2 (+3.32),"[10.778, -9.242, 3.336]",13/2/2021,"[[867418, Nivel 1 (+0.00), P2R - 900, Puertas]..."
2,Conflicto2,2.- ARQ_PUE VS EST_files\cd010002.jpg,-0.116,Activo,3'-C : Nivel 3 OFI (+5.90),"[10.838, -9.358, 6.440]",13/2/2021,"[[1654802, Nivel 2 AUL (+3.60), P2R - 900, Pue..."
3,Conflicto3,2.- ARQ_PUE VS EST_files\cd010003.jpg,-0.099,Activo,8a-D : Nivel 4 OFI (+8.85),"[34.666, -12.929, 10.100]",13/2/2021,"[[1244315, Nivel 3 - AUL (+7.20), Aluminio, Só..."
4,Conflicto4,2.- ARQ_PUE VS EST_files\cd010004.jpg,-0.098,Activo,8a-D : Nivel 3 OFI (+5.90),"[34.671, -12.929, 6.440]",13/2/2021,"[[1237349, Nivel 2 AUL (+3.60), Aluminio, Sóli..."
5,Conflicto5,2.- ARQ_PUE VS EST_files\cd010005.jpg,-0.078,Activo,8a-D : Nivel 3 (+6.58),"[34.671, -13.092, 6.940]",13/2/2021,"[[1237433, Nivel 2 AUL (+3.60), Aluminio, Sóli..."
6,Conflicto6,2.- ARQ_PUE VS EST_files\cd010006.jpg,-0.064,Activo,8a-D : Nivel 4 OFI (+8.85),"[34.711, -13.066, 10.100]",13/2/2021,"[[1244273, Nivel 3 - AUL (+7.20), P2M - 1200, ..."
7,Conflicto7,2.- ARQ_PUE VS EST_files\cd010007.jpg,-0.054,Activo,8a-B : Nivel 4 OFI (+8.85),"[33.880, -4.904, 10.040]",13/2/2021,"[[1243682, Nivel 3 - AUL (+7.20), Aluminio, Só..."
8,Conflicto8,2.- ARQ_PUE VS EST_files\cd010008.jpg,-0.050,Activo,D-3 : Nivel 5 (+11.86),"[7.108, -12.893, 13.300]",13/2/2021,"[[1703328, Nivel 4 - AUL (+10.80), Aluminio, S..."
9,Conflicto9,2.- ARQ_PUE VS EST_files\cd010009.jpg,-0.036,Activo,F''-0 : Nivel 2 (+1.62),"[-10.230, -23.483, 2.373]",13/2/2021,"[[996102, Nivel 1 (+0.00), Aluminio, Sólido], ..."
10,Conflicto11,2.- ARQ_PUE VS EST_files\cd010010.jpg,-0.030,Activo,J-9 : Nivel 3 - AUL (+7.20),"[37.395, -40.847, 8.350]",13/2/2021,"[[1241752, Nivel 3 OFI (+5.90), P1M - 90, Part..."


## Dataframe a PDF

In [58]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import BaseDocTemplate, Frame, Paragraph, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from io import BytesIO

def generate_pdf(dataframe, pdf_output):
    pdf_buffer = BytesIO()
    pdf = BaseDocTemplate(pdf_buffer, pagesize=letter)

    styles = getSampleStyleSheet()
    style_normal = styles['Normal']

    # Lista para almacenar los elementos de cada página
    elements = []

    for index, row in dataframe.iterrows():
        # Convertir la fila en una lista de listas para la tabla
        data = [list(row)]

        # Crear una tabla a partir de los datos
        table = Table(data, style=[('GRID', (0, 0), (-1, -1), 1, colors.black)])

        # Crear un marco para la tabla
        frame = Frame(pdf.leftMargin, pdf.bottomMargin, pdf.width, pdf.height - 2 * pdf.bottomMargin, id=f'frame_{index}')
        frame.add(table)

        # Agregar el marco al PDF
        pdf.addPageTemplates([frame])

        # Agregar la tabla a la lista de elementos
        elements.append(Paragraph(f"<b>Conflicto {index + 1}</b>", style_normal))  # Añadir un título

    # Construir el PDF con todas las páginas
    pdf.build(elements)

    # Guardar el PDF en un archivo
    with open(pdf_output, 'wb') as f:
        f.write(pdf_buffer.getvalue())

# Llamar a la función con tu DataFrame y la ruta de salida del PDF
generate_pdf(archivo_df, 'output.pdf')


TypeError: Frame._add() missing 1 required positional argument: 'canv'

# Ejemplo de listas


In [ ]:
def generar_pdf(xml_path, pdf_output):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    c = canvas.Canvas(pdf_output, pagesize=letter)
    width, height = letter

    contador_conflictos = 1
    nombres_conflictos = []
    imagenes = []
    
    




    for clashresult in root.findall('.//clashresult'):
        nombre_conflicto = f"Conflicto {contador_conflictos}"
        nombres_conflictos.append(nombre_conflicto)
        imagen_filename = clashresult.get('href')
        imagenes.append(imagen_filename)
        distancia = clashresult.get('distance')
        clashpoint = clashresult.find('.//clashpoint/pos3f')
        x, y, z = clashpoint.get('x'), clashpoint.get('y'), clashpoint.get('z')
        gridlocation = clashresult.find('gridlocation').text

# FUNCION GENERAR PDF

In [ ]:
def generar_pdf(xml_path, pdf_output):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    c = canvas.Canvas(pdf_output, pagesize=letter)
    width, height = letter

    contador_conflictos = 1
    Nombre_reporte = []
    Tolerancia = []
    Interferencias_totales = []
    Interferencias_abiertas = []
    Interferencias_resueltas = []
    Nombres_confictos = []
    Imagenes_reportes = []
    Estados_interferecnias = []
    Distancias_interferencias = []
    Ejes_X = []
    Ejes_Y = []
    Ejes_Z = []
    Nivel_interferencia = []
    Años = []
    Meses = []
    Dias = []
    Nombre_informe = []
    ID_elemento_1 = []
    Nivel_elemento_1 = []
    Nombre_elemento_1 = []
    Valor_nombre_elemento_1 = []
    Tipo_elemento_1 = []
    Tipo_nombre_elemento_1 = []
    ID_elemento_2 = []
    Nivel_elemento_2 = []
    Nombre_elemento_2 = []
    Valor_nombre_elemento_2 = []
    Tipo_elemento_2 = []
    Tipo_nombre_elemento_2 = []


    for clashtest in root.find('.//clashtest'):
        nombre_todo = clashtest.get('name')
        Nombre_reporte.append(nombre_todo)
        tolerancia = clashtest.get('tolerance')
        Tolerancia.append(tolerancia)
        summary = clashtest.find('.//summary')
        interferencias_totales = summary.get('total')
        Interferencias_totales.append('interferencias_totales')
        interferencias_abiertas = summary.get('active')
        Interferencias_abiertas.append('interferencias_abiertas')
        interferencias_resueltas = summary.get('resolved')
        Interferencias_resueltas.append('interferencias_resueltas')
      for clashresult in root.findall('.//clashresult'):
          nombre_conflicto = f"Conflicto {contador_conflictos}"
          Nombres_confictos.append('nombre_conflicto')
          imagen_filename = clashresult.get('href')
          Imagenes_reportes.append('imagen_filename')
          estado_interferencia = clashresult.get('status')
          Estados_interferecnias.append('estado_interferencia')
          distancia = clashresult.get('distance')
          Distancias_interferencias.append('distancia')
          clashpoint = clashresult.find('.//clashpoint/pos3f')
          x, y, z = clashpoint.get('x'), clashpoint.get('y'), clashpoint.get('z')
          Ejes_X.append('x')
          Ejes_Y.append('y')
          Ejes_Z.append('z')
          gridlocation = clashresult.find('.//gridlocation').text
          nivel = gridlocation.cdata
          Nivel_interferencia.append('nivel')
          date = clashresult.find('.//createddate/date')
          año, mes, día = date.get('year'), date.get('month'),date.get('day')
          Años.append('año')
          Meses.append('mes')
          Dias.append('día')
          for clashobjects in root.find('.//clashobject'[0]):
            id1 = objectattribute.value.cdata
            ID_elemento_1.append('id1')
            nivel1 = layer.cdata
            Nivel_elemento_1.append('nivel1')
            nombre1.1 = smarttags.smarttag[0].name.cdata
            Nombre_elemento_1.append('nombre1.1')
            tipo1.1 = smarttags.smarttag[0].value.cdata
            Valor_nombre_elemento_1.append('tipo1.1')
            nombre1.2 = smarttags.smarttag[1].name.cdata
            Tipo_elemento_1.append('nombre1.2')
            tipo1.2 = smarttags.smarttag[1].value.cdata
            Tipo_nombre_elemento_1.append('tipo1.2')
          for clashobjects in root.find('.//clashobject'[1]):
            id2 = objectattribute.value.cdata
            ID_elemento_2.append('id2')
            nivel2 = layer.cdata
            Nivel_elemento_2.append('nivel2')
            nombre2.1 = smarttags.smarttag[0].name.cdata
            Nombre_elemento_2.append('nombre2.1')
            tipo2.1 = smarttags.smarttag[0].value.cdata
            Valor_nombre_elemento_2.append('tipo2.1')
            nombre2.2 = smarttags.smarttag[1].name.cdata
            Tipo_elemento_2.append('nombre2.2')
            tipo2.2 = smarttags.smarttag[1].value.cdata
            Tipo_nombre_elemento_2.append('tipo2.2')


        # Agregar detalles al PDF
        c.drawString(10, height - 50, f"Nombre del conflicto: {nombre_conflicto}")
        c.drawString(10, height - 70, f"Distancia: {distancia}")
        c.drawString(10, height - 90, f"Coordenadas del punto de conflicto: ({x}, {y}, {z})")
        c.drawString(10, height - 110, f"Ubicación en la cuadrícula: {gridlocation}")

        # Obtener la imagen asociada
        imagen_path = os.path.join(os.path.dirname(xml_path), imagen_filename.replace('\\', '/'))
        imagen = utils.ImageReader(imagen_path)
        c.drawImage(imagen, 10, height - 250, width=400, height=200)

        # Agregar un salto de página para la próxima interferencia
        c.showPage()

        contador_conflictos += 1

    c.save()
    return nombres_conflictos, nombres_conflictos, nombres_conflictos

xml_path = '/content/2.- ARQ_PUE VS EST.xml'
pdf_output = '/content/downloads/resultado.pdf'

generar_pdf(xml_path, pdf_output)

SyntaxError: ignored

# VARIABLES PARA LA FUNCION

In [ ]:
xml_path = '/content/2.- ARQ_PUE VS EST.xml'
pdf_output = '/content/downloads/resultado.pdf'


# EJECUTAR FUNCION

In [ ]:
generar_pdf(xml_path, pdf_output)